In [23]:
import json
import pandas as pd

In [24]:
# https://cloud.google.com/compute/docs/general-purpose-machines#n2-high-cpu
file = './data/ep_ce_instances.json'
f = open(file, 'r')
data = json.load(f)['data']
machine_types = {
    "f1-micro": {
        'vCPUs': 1,
        "memory_gb": 0.6,
        "bandwidth_gbps": 1
    },
    "g1-small": {
        'vCPUs': 1,
        "memory_gb": 1.7,
        "bandwidth_gbps": 1
    },
    "n2-highcpu-2": {
        'vCPUs': 2,
        "memory_gb": 2,
        "bandwidth_gbps": 10
    },
    "e2-micro": {
        'vCPUs': 2,
        "memory_gb": 1,
        "bandwidth_gbps": 1
    },
    "e2-small": {
        'vCPUs': 2,
        "memory_gb": 2,
        "bandwidth_gbps": 1
    },
}
disk_type_enum = {
    'PERSISTENT': 0
}

In [25]:
len(data)

2531

In [26]:
gcp_data = list(filter(lambda x: x.get('provider') == 'GCP', data))

In [27]:
len(gcp_data)

190

In [32]:
ep_gcp_map = {}
for i in range(0, len(gcp_data)):
    variant = gcp_data[i]['variant']
    if variant.get('id'):
        instance_type = variant["machineType"].split('/')[-1]
        ep_gcp_map[variant['id']] = {
            "id": variant['id'],
            "disk_type": list(map(lambda x: x.get('type'), variant['disks']))[0],
            "instance_type": instance_type,
            "region": variant.get("region"),
            "vCPUs": machine_types[instance_type]["vCPUs"],
            "memory_gb": machine_types[instance_type]["memory_gb"],
            "bandwidth_gbps": machine_types[instance_type]["bandwidth_gbps"]
        }

In [33]:
ep_gcp_map

{'2265529010543344118': {'id': '2265529010543344118',
  'disk_type': 'PERSISTENT',
  'instance_type': 'g1-small',
  'region': None,
  'vCPUs': 1,
  'memory_gb': 1.7,
  'bandwidth_gbps': 1},
 '5353556143556258917': {'id': '5353556143556258917',
  'disk_type': 'PERSISTENT',
  'instance_type': 'g1-small',
  'region': None,
  'vCPUs': 1,
  'memory_gb': 1.7,
  'bandwidth_gbps': 1},
 '3922265808357194945': {'id': '3922265808357194945',
  'disk_type': 'PERSISTENT',
  'instance_type': 'f1-micro',
  'region': None,
  'vCPUs': 1,
  'memory_gb': 0.6,
  'bandwidth_gbps': 1},
 '4036654423970744204': {'id': '4036654423970744204',
  'disk_type': 'PERSISTENT',
  'instance_type': 'g1-small',
  'region': None,
  'vCPUs': 1,
  'memory_gb': 1.7,
  'bandwidth_gbps': 1},
 '1240637607697652238': {'id': '1240637607697652238',
  'disk_type': 'PERSISTENT',
  'instance_type': 'f1-micro',
  'region': None,
  'vCPUs': 1,
  'memory_gb': 0.6,
  'bandwidth_gbps': 1},
 '6856595969022927546': {'id': '685659596902292754

In [44]:
gcp_ce_file = './data/gcp_cost.csv'
df_cost = pd.read_csv(gcp_ce_file, header=0, sep=',', dtype =  {"gcpId": str})

In [45]:
df_cost

,Unnamed: 0,cost,date,costInUsd,gcpId,instanceType,disk
0,0,0.953786,2022-07-04,0.953786,6762571275092817403,g1-small,PERSISTENT
1,1,0.839175,2022-07-04,0.839175,7994828870141789187,g1-small,PERSISTENT
2,2,0.211182,2022-09-07,0.211182,2084983531904533635,g1-small,PERSISTENT
3,3,0.324464,2022-09-07,0.324464,1037551659627754032,f1-micro,PERSISTENT
4,4,0.326841,2022-09-07,0.326841,1774086229609597003,f1-micro,PERSISTENT
...,...,...,...,...,...,...,...
5637,5637,0.399614,2022-09-14,0.399614,8842805660917525667,f1-micro,PERSISTENT
5638,5638,0.834418,2022-08-30,0.834418,7440776697402117547,g1-small,PERSISTENT
5639,5639,0.142453,2022-07-27,0.142453,2332843410177307106,f1-micro,PERSISTENT
5640,5640,2.482112,2022-08-01,2.482112,3838479753825543664,g1-small,PERSISTENT


In [59]:
# df['gcpId'] = df.apply(lambda row: str(ep_gcp_map.get(row.epId)), axis=1)
df_new = df_cost
def extract_variant(row, name):
    record = ep_gcp_map.get(row.gcpId)
    if not record:
        return ''
    return record.get(name)
df_new['instance_type'] = df_new.apply(lambda row: extract_variant(row, 'instance_type'), axis=1)
df_new['disk'] = df_new.apply(lambda row: extract_variant(row, 'disk_type'), axis=1)
df_new['vCPUs'] = df_new.apply(lambda row: extract_variant(row, 'vCPUs'), axis=1)
df_new['memory_gb'] = df_new.apply(lambda row: extract_variant(row, 'memory_gb'), axis=1)
df_new['bandwidth_gbps'] = df_new.apply(lambda row: extract_variant(row, 'bandwidth_gbps'), axis=1)
df_new['instanceId'] = df_new['gcpId']
df_new['datetime'] = df_new['date']

In [60]:
df_filtered = df_new.loc[df_new['gcpId'] != None]

In [61]:
df_filtered[['cost', 'datetime', 'costInUsd', 'instanceId', 'instance_type', 'disk', 'vCPUs', 'memory_gb', 'bandwidth_gbps']].to_csv('./data/gcp_cost_with_instance_info.csv')